In [ ]:
# Imports
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os

from theme_attention.rnd import marketsTool as tools

import warnings

from base.sql import factset as fs
from theme_attention.pipeline import dbTool as dbt 

import yaml
from pathlib import Path
import utils
import importlib 
importlib.reload(utils) 

import requests
from io import StringIO
import yfinance as yf
import pickle

In [ ]:
# Adjust Pandas display settings to show all rows and columns
#pd.set_option('display.max_rows', None)  # Show all rows
#pd.set_option('display.max_columns', None)  # Show all columns
#pd.set_option('display.width', 1000)  # Adjust the width to fit the output

# Reset Display Settings
#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_columns')
#pd.reset_option('display.width')

In [ ]:
# --- TICKERS (initial idea, not used at the moment) ---
headers = {"User-Agent": "Mozilla/5.0"}

# Fetch HTML content
html = requests.get(
    "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    headers=headers
).text

# Read tables
tables = pd.read_html(StringIO(html))
sp500 = tables[1]

# Extract raw tickers
tickers_500 = sp500["Symbol"].tolist()

# Tickers of the top 20 companies by market capitalization in the S&P 500
# If "tickers_500" is too big to process, use "tickers_20" instead
tickers_20 = ["NVDA","AAPL","MSFT","AMZN","GOOGL","AVGO","GOOG","META","TSLA","BRK.B","LLY","JPM","V","XOM","JNJ","WMT","NFLX","MA","ABBV","COST"]

# Bloomberg tickers (US equities)
bb_tickers = [
    f"{t.replace('.', '/').replace(':', '/').replace('-', '/')} US"
    for t in tickers_20
]

# Map Yahoo exchange codes to readable names
yahoo_to_exch = {
    "NMS": "NASDAQ",  
    "NGM": "NASDAQ",
    "NGS": "NASDAQ",
    "NSQ": "NASDAQ",
    "NYQ": "NYSE",    
    "NYS": "NYSE",
    "PCX": "AMEX",    
}

def get_exchange(symbol):
    t = yf.Ticker(symbol)
    ex = t.info.get("exchange")  # e.g. "NMS", "NYQ"
    if ex is None:
        return "UNKNOWN"
    return yahoo_to_exch.get(ex, ex)  # map if known, else keep raw

# Build market tickers: EXCHANGE:SYMBOL (e.g. NASDAQ:AAPL)
exchanges = [get_exchange(sym) for sym in tickers_20]
mkt_tickers = [f"{ex}:{sym}" for ex, sym in zip(exchanges, tickers_20)]

# Create dataframe
df = pd.DataFrame({
    "symbol": tickers_20,
    "bb_ticker": bb_tickers,
    "mkt_ticker": mkt_tickers
})

df


In [ ]:
# Adjust the mkt_ticker for Berkshire Hathaway manually
df.loc[df["symbol"] == "BRK.B", "mkt_ticker"] = "NYSE:BRK.B"

df

In [ ]:
# Call existing ticker mapping function to get fsym_id and other info
mapT = fs.getFsymIDsFromBBTicker(bb_tickers)

masterT = fs.getFactsetCoverage(mapT['fsym_id'].to_frame())
masterT = masterT.merge(mapT, on='fsym_id')

# Merge Bloomberg ticker, and FactSet ID into one DataFrame
companiesDB = df[['bb_ticker', 'mkt_ticker']].merge(mapT, left_on='bb_ticker', right_on='bbg_ticker', how='left').merge(masterT, left_on='fsym_id', right_on='fsym_id', how='left')

# Final 
companiesMapping = companiesDB[['bb_ticker', 'mkt_ticker', 'fsym_id', 'proper_name']].drop_duplicates().reset_index(drop=True)

companiesMapping


In [ ]:
# --- TICKERS ---
# Region Codes (EOD Mapping)
# US  -> United States
# RDM -> Rest of Developed Markets (e.g., Europe, Japan, Australia, etc.)
# EM  -> Emerging Markets (e.g., China, India, Brazil, etc.)

fL = dbt.get_master_mapping_df(region=['US'])
# len(fL["sub_industry"].unique())
fL

In [ ]:
# Convert exit_date to datetime
fL["exit_date"] = pd.to_datetime(fL["exit_date"], errors="coerce")

# Keep only rows where exit_date is in the future
today = pd.Timestamp.today().normalize()
mask_date = fL["exit_date"] >= today

# Apply filter
fL_filtered = fL[mask_date].copy()

fL_filtered

In [ ]:
# Keep only needed columns
companiesMapping = fL_filtered[
    ['fsym_id', 'proper_name', 'price_id', 'Code', 'bbg_ticker']
].copy()

companiesMapping

In [ ]:
# Get market capitalizations of the companies
preliminary_fsym_ids = companiesMapping['fsym_id'].unique().tolist()

market_cap = fs.getFundamentals(preliminary_fsym_ids, 'ff_mkt_val', 0, datetime.today() - relativedelta(months=3) , datetime.today())

market_cap

In [ ]:
# Merge market capitalization into companiesMapping
companiesMapping = companiesMapping.merge(
    market_cap[['fsym_id', 'qf']],
    on='fsym_id',
    how='left'
)
companiesMapping = companiesMapping.rename(columns={'qf': 'mkt_cap'}).dropna()

companiesMapping

In [ ]:
FILE_PATH = "/home/azureuser/cloudfiles/code/Users/manuel.noseda/DL_project_news/WatchListNews_FULL_PERIOD.pkl"

with open(FILE_PATH, "rb") as f:
    news_dict = pickle.load(f)

news_dict

In [ ]:
# keep only companies whose Code exists in news_dict
companies_with_news = companiesMapping[
    companiesMapping['Code'].isin(news_dict.keys())
]

# Get the top 20 companies by market capitalization (and drop the mkt_cap column which is no more needed for cleaner output)
top500 = (
    companies_with_news
    .sort_values('mkt_cap', ascending=False)
    .head(500)
    .drop(columns='mkt_cap')
    .reset_index(drop=True)
)

top500

In [ ]:
fsym_ids = top500['fsym_id'].dropna().tolist()
price_ids = top500['price_id'].dropna().tolist()
eod_codes = top500['Code'].dropna().tolist()

In [ ]:
# Build the IDdf exactly as getAdjustedPrices() expects
IDdf = pd.DataFrame({
    "fsym_id": fsym_ids,
    "price_id": price_ids   
})

IDdf

In [ ]:
# Format: year, month, day

startDate = datetime(2024,10,1) 
endDate = datetime(2025,8,1)
# Get raw prices
df_prices = fs.getAdjustedPrices(IDdf, startDate, endDate)


In [ ]:
# Select only needed columns from prices
df_small = df_prices[["fsym_id", "date", "adj_price", "unadj_price"]]

# Build the final dictionary
companies_dict = {}

# Store companyMapping and prices for each fsym_id
for fsym, grp in df_small.groupby("fsym_id"):
    companies_dict[fsym] = {
        "companyMapping": top500[top500["fsym_id"] == fsym].copy(),
        "prices": grp.drop(columns="fsym_id").reset_index(drop=True)
    }

# Access the DataFrame of prices for the first fsym_id
first_id = fsym_ids[0]
proper_name = companies_dict[first_id]['companyMapping']['proper_name'].iloc[0]
print(f'Prices for company "{proper_name}":')
companies_dict[first_id]["prices"]

config_path = Path.cwd() / "config" / "DL_project_config.yml"
with open(config_path) as f:
    config = yaml.safe_load(f)

In [ ]:
# Fundamentals to keep
fundamentals_keep = [
    'eps_basic',
    'eps_dil',
    'assets',
    'shldrs_eq',
    'inven',
    'cash_st',
    'debt',
    'net_debt',
    'net_inc',
    'oper_cf'
]

# Iterate over each fsym_id, fetch fundamentals, filter (drop those used for ratios), then store
for fsym_id in fsym_ids:
    # 1) Get full fundamentals dict for this fsym_id
    fundamentals_raw = utils.get_fundamentals(fsym_id, startDate, endDate, config)

    # 2) Keep only the desired keys
    fundamentals_filtered = {
        key: df.drop(columns=["fsym_id"], errors="ignore").reset_index(drop=True)
        for key, df in fundamentals_raw.items()
        if key in fundamentals_keep
    }
    
    # 3) Ensure entry exists and assign filtered fundamentals
    if fsym_id not in companies_dict:
        companies_dict[fsym_id] = {}

    companies_dict[fsym_id]["fundamentals"] = fundamentals_filtered

# Access the DataFrame of fundamentals for the first fsym_id
first_id = fsym_ids[0]
proper_name = companies_dict[first_id]['companyMapping']['proper_name'].iloc[0]
print(f'Fundamentals for company "{proper_name}":')
companies_dict[first_id]["fundamentals"]

In [ ]:
# Iterate over each fsym_id and compute ratios
for fsym_id in fsym_ids:
    df = utils.get_ratios(fsym_id, startDate, endDate, config)

    # Convert index to a column named "date"
    df = df.copy()
    df["date"] = df.index
    df = df.reset_index(drop=True)

    # Put "date" as the first column
    cols = ["date"] + [c for c in df.columns if c != "date"]
    df = df[cols]

    companies_dict[fsym_id]["ratios"] = df

# Access the DataFrame of ratios for the first fsym_id
first_id = fsym_ids[0]
proper_name = companies_dict[first_id]['companyMapping']['proper_name'].iloc[0]
print(f'Ratios for company "{proper_name}":')
companies_dict[first_id]["ratios"]

In [ ]:
# Iterate over each fsym_id and retrieve news
for fsym_id in fsym_ids:

    company_data = companies_dict[fsym_id]['companyMapping']
    eod_code = company_data['Code'].iloc[0]

    news = news_dict[eod_code]

    companies_dict[fsym_id]["news"] = news

# Access the DataFrame of news for the first fsym_id
first_id = fsym_ids[0]
proper_name = companies_dict[first_id]['companyMapping']['proper_name'].iloc[0]
print(f'News for company "{proper_name}":')
companies_dict[first_id]["news"]

In [ ]:
# Iterate over each fsym_id and retrieve earning announcements dates 
for fsym_id in fsym_ids:

    company_data = companies_dict[fsym_id]['companyMapping']
    eod_code = company_data['Code'].iloc[0]
    symbol = eod_code.split(".")[0]
    ticker = yf.Ticker(symbol)

    earnings = ticker.get_earnings_dates(limit=20)

    if earnings is None or earnings.empty:
        print(f"No earnings data for {symbol}")
        continue

    # Keep only earnings announcements
    earnings = earnings[earnings["Event Type"] == "Earnings"]

    # Convert index to a column named "Earnings Date"
    earnings = earnings.copy()
    earnings["Earnings Date"] = earnings.index
    earnings = earnings.reset_index(drop=True)

    # Put "Earnings Date" as the first column
    cols = ["Earnings Date"] + [c for c in earnings.columns if c != "Earnings Date"]
    earnings = earnings[cols]

    companies_dict[fsym_id]["earnings"] = earnings

    

In [ ]:
# Access the DataFrame of earnings for the first fsym_id (done separately to get the output if there are no earnings data for a company in the previous cell)
first_id = fsym_ids[0]
proper_name = companies_dict[first_id]['companyMapping']['proper_name'].iloc[0]
print(f'Earnings for company "{proper_name}":')
companies_dict[first_id]["earnings"]

In [ ]:
# Save the dictionary to a pickle file

local_path = '/home/azureuser/cloudfiles/code/Users/manuel.noseda/temp'
file_name_root = 'DL_dataset'

# create full file path
file_path = os.path.join(local_path, f"{file_name_root}.pkl")

# save the dictionary
with open(file_path, "wb") as f:
    pickle.dump(companies_dict, f)

print(f"File saved in: {file_path}")


In [ ]:
'''

DATASET STRUCTURE

companies_dict
│
├── fsym_id_1
│     ├── companyMapping     → DataFrame
│     ├── prices             → DataFrame
│     ├── fundamentals       → dict of DataFrames
│     ├── ratios             → DataFrame
│     ├── news               → DataFrame
│     └── earnings           → DataFrame
│
├── fsym_id_2
│     ├── companyMapping
│     ├── prices
│     ├── fundamentals
│     ├── ratios
│     ├── news
│     └── earnings
│
└── ...

companyMapping : DataFrame
    ├── fsym_id
    ├── proper_name
    ├── price_id
    ├── Code
    └── bbg_ticker

prices : DataFrame
    ├── date          
    ├── adj_price     
    └── unadj_price  

fundamentals: Dictionary
    ├── eps_basic   → DataFrame
    ├── eps_dil     → DataFrame
    ├── assets      → DataFrame
    ├── shldrs_eq   → DataFrame
    ├── inven       → DataFrame
    ├── cash_st     → DataFrame
    ├── debt        → DataFrame
    ├── net_debt    → DataFrame
    ├── net_inc     → DataFrame
    └── oper_cf     → DataFrame

    Each of these DataFrames has the same structure: date | af | qf | saf/ltm

ratios : DataFrame
    ├── date
    ├── Dividend_Yield
    ├── Net_Margin
    ├── Gross_Margin
    ├── ROE
    ├── ROA
    └── Debt_to_Equity

news : DataFrame
    ├── id
    ├── event_type
    ├── event_time 
    ├── msh_ids
    ├── event_data 
    └── tags

earnings : DataFrame
    ├── Earnings Date
    ├── EPS Estimate
    ├── Reported EPS
    ├── Surprise(%)
    └── Event Type

'''